In [17]:
# Load and inspect the features table / Change the name in the code below if needed.
# (Optional) change the features_quantification_matrix.csv to the name of your .csv file.
# Make sure to add the value of bioactivity in the second columns
tab <- read.csv("features_quantification_matrix_edited_bioactivity.csv")
nm <- as.vector(as.matrix(read.csv("features_quantification_matrix_edited_bioactivity.csv", nrow=1, header=FALSE)))
                                                                                                 
colnames(tab) <- nm 
dim(tab)                  
tab[1:5,1:5]                                                       


[1]  14 589

Sample name,BioactivityCHIKV,270.279 1698 1 (ID: 9),271.283 1699 1 (ID: 355),279.174 1413 1 (ID: 511)
Extract,68,5690331,569378.1,3172924
F_5,1,246596026,40396489.5,0
F_6,4,186949612,30710029.2,0
F_7,1,362958584,63895441.0,0
F_8,3,160968307,26317669.9,0


In [18]:
# Take out blank rows in the table
tab <- tab[!is.na(tab[,2]),]

In [19]:
# Normalize the features by TIC and adding 1 to all feature intensities 
# to help scaling
tab2 <- tab
tab2[,-c(1:2)] <- t(apply(tab2[,-c(1:2)], 1, function(x) (x+1)/sum((x+1))))
#tab2[,-c(1:2)] <- t(apply(tab2[,-c(1:2)], 1, function(x) (x+1)))

In [20]:
# Calculate the correlation coefficient between a single feature and the bioactivity.
# Scale should help correlation - https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1534033/
cor.test(scale(tab2[,2])[,1], scale(tab2[,3])[,1])[c("estimate", "p.value")]

$estimate
       cor 
-0.3243776 

$p.value
[1] 0.2578494

In [21]:
# How to do for all features
ct <- t(sapply(3:ncol(tab2), function(x) unlist(cor.test(scale(tab2[,2])[,1], scale(tab2[,x])[,1])[c("estimate", "p.value")])))

In [22]:
# Show the dimensions of the features_quantificaton_matrix.csv
dim(tab2)
dim(ct)


[1]  14 589

[1] 587   2

In [23]:
# Create an output table with correlation coefficient value and p-value for every features

ct <- rbind(c("cor"," p_value"), c(0,0), ct)

tab3 <- rbind(t(ct),  as.matrix(tab2))
rownames(tab3) <- NULL
tab3[1:5, 1:5]
write.csv(tab3, "features_quantification_matrix_edited_with_correlation.csv", row.names=FALSE)

Sample name,BioactivityCHIKV,270.279 1698 1 (ID: 9),271.283 1699 1 (ID: 355),279.174 1413 1 (ID: 511)
cor,0,-0.324377606725705,-0.318374649942669,0.11344978411662
p_value,0,0.257849356719369,0.267271199671765,0.699375059832539
Extract,68.0,0.0001391824,0.0000139267,7.760803e-05
F_5,1.0,0.1594079650,0.0261136500,6.464336e-10
F_6,4.0,0.0228397314,0.0037518603,1.221705e-10


In [24]:
# Tranpose the table for molecular networking mapping in Cytoscape
new = t(tab3)
colnames(new) = new[1,]
new = new[-1,]
new = cbind(0:(nrow(new)-1), rownames(new), new)
rownames(new) <- NULL
colnames(new)[1:2] <- c("shared name", "IDs")
new[1,1] <- ""
new[1:5,1:5]
write.csv(new, "features_quantification_matrix_transposed_with_correlation.csv", row.names=FALSE)

shared name,IDs,cor,p_value,Extract
,BioactivityCHIKV,0,0,68.0
1,270.279 1698 1 (ID: 9),-0.324377606725705,0.257849356719369,0.0001391824
2,271.283 1699 1 (ID: 355),-0.318374649942669,0.267271199671765,0.0000139267
3,279.174 1413 1 (ID: 511),0.11344978411662,0.699375059832539,7.760803e-05
4,280.264 1420 1 (ID: 1029),-0.200702959079237,0.49145327312603,4.456230e-05


In [25]:
# Get the significant correlation coefficients for both cases (>0.05)
which(as.numeric(ct[-c(1,2),2])<0.05)

[1]  17  27  29  30  94  99 100 108 115 120 137 138 157 158 162 171 183 201 202
[20] 207 210 239 241 255 285 292 299 306 323 329 331 346 357 360 366 367 370 371
[39] 390 416 431 433 440 446 461 462 467 469 476 482 487 496 504 512 513 518 524
[58] 531 543 544 545 556

In [26]:
# Show the features ID with correlation coefficient
nm[-c(1:2)][as.numeric(ct[-c(1,2),2])<0.05]

[1] "383.222 1524 1 (ID: 259)" "401.232 1290 1 (ID: 171)"
 [3] "401.232 1619 1 (ID: 23)"  "401.232 1527 1 (ID: 7)"  
 [5] "563.296 1291 1 (ID: 164)" "571.255 559 1 (ID: 626)" 
 [7] "571.301 1719 1 (ID: 50)"  "577.265 998 1 (ID: 194)" 
 [9] "589.311 1520 1 (ID: 22)"  "591.326 1626 1 (ID: 142)"
[11] "613.266 824 1 (ID: 107)"  "613.266 874 1 (ID: 372)" 
[13] "641.294 836 1 (ID: 375)"  "641.294 742 1 (ID: 100)" 
[15] "647.307 997 1 (ID: 157)"  "661.287 945 1 (ID: 163)" 
[17] "669.289 697 1 (ID: 154)"  "681.290 558 1 (ID: 214)" 
[19] "681.290 742 1 (ID: 106)"  "683.303 823 1 (ID: 3)"   
[21] "684.310 823 1 (ID: 5)"    "701.314 827 1 (ID: 198)" 
[23] "703.290 730 1 (ID: 133)"  "709.284 697 1 (ID: 155)" 
[25] "721.303 945 1 (ID: 159)"  "723.299 822 1 (ID: 13)"  
[27] "725.339 774 1 (ID: 165)"  "730.340 898 1 (ID: 38)"  
[29] "737.310 963 1 (ID: 83)"   "739.319 785 1 (ID: 101)" 
[31] "739.319 593 1 (ID: 91)"   "747.321 773 1 (ID: 134)" 
[33] "753.332 773 1 (ID: 625)"  "757.285 870 1 (ID: 223)" 
[35] "758.373 1047 1 (ID: 144)" "758.373 1257 1 (ID: 615)"
[37] "761.301 805 1 (ID: 110)"  "761.301 595 1 (ID: 176)" 
[39] "767.345 870 1 (ID: 14)"   "780.353 1045 1 (ID: 616)"
[41] "789.332 870 1 (ID: 34)"   "789.332 1014 1 (ID: 108)"
[43] "792.357 1201 1 (ID: 32)"  "793.361 1211 1 (ID: 378)"
[45] "802.398 1055 1 (ID: 137)" "803.307 939 1 (ID: 24)"  
[47] "806.335 1158 1 (ID: 111)" "806.335 1061 1 (ID: 31)" 
[49] "814.337 1192 1 (ID: 293)" "817.321 1105 1 (ID: 291)"
[51] "820.349 1210 1 (ID: 180)" "824.377 1038 1 (ID: 619)"
[53] "828.318 1067 1 (ID: 210)" "834.368 1269 1 (ID: 76)" 
[55] "834.368 1463 1 (ID: 620)" "836.384 1150 1 (ID: 161)"
[57] "840.319 1120 1 (ID: 70)"  "844.411 1331 1 (ID: 73)" 
[59] "856.348 1463 1 (ID: 611)" "856.348 1245 1 (ID: 643)"
[61] "858.369 1147 1 (ID: 622)" "866.393 1330 1 (ID: 286)"

In [27]:
which(as.numeric(ct[-c(1,2),2])<0.05)

[1]  17  27  29  30  94  99 100 108 115 120 137 138 157 158 162 171 183 201 202
[20] 207 210 239 241 255 285 292 299 306 323 329 331 346 357 360 366 367 370 371
[39] 390 416 431 433 440 446 461 462 467 469 476 482 487 496 504 512 513 518 524
[58] 531 543 544 545 556

In [28]:
# Table with significant correlation coefficients. Using Bonferroni method.
ct[-c(1,2),][which(p.adjust(as.numeric(ct[-c(1:2),2]), method = "bonferroni")<0.05),]

estimate.cor,p.value
0.911533897243834,5.70642706988177e-06
0.878799706808992,3.50720064572382e-05
0.907554668668843,7.36501900414158e-06
0.905633511042207,8.29688606983179e-06
0.919233388225548,3.36127974167346e-06
0.873397143507255,4.50107734106115e-05
0.911780588059994,5.61468814684507e-06
0.910238252830603,6.20871030421146e-06
0.913168025609065,5.12101071510364e-06
0.924118681813123,2.33675278160981e-06


In [29]:
# Call the ID
which(p.adjust(as.numeric(ct[-c(1:2),2]), method = "bonferroni")<0.05)

[1]  27  30  94  99 108 115 120 137 138 157 162 171 183 201 207 210 239 241 255
[20] 285 299 331 346 357 360 366 367 371 390 416 431 433 446 461 462 469 487 496
[39] 504 513 518 524 543 544 545 556

In [30]:
# Features passing Bonferronii method
nm[-c(1:2)][which(p.adjust(as.numeric(ct[-c(1:2),2]), method = "bonferroni")<0.05)]


[1] "401.232 1290 1 (ID: 171)" "401.232 1527 1 (ID: 7)"  
 [3] "563.296 1291 1 (ID: 164)" "571.255 559 1 (ID: 626)" 
 [5] "577.265 998 1 (ID: 194)"  "589.311 1520 1 (ID: 22)" 
 [7] "591.326 1626 1 (ID: 142)" "613.266 824 1 (ID: 107)" 
 [9] "613.266 874 1 (ID: 372)"  "641.294 836 1 (ID: 375)" 
[11] "647.307 997 1 (ID: 157)"  "661.287 945 1 (ID: 163)" 
[13] "669.289 697 1 (ID: 154)"  "681.290 558 1 (ID: 214)" 
[15] "683.303 823 1 (ID: 3)"    "684.310 823 1 (ID: 5)"   
[17] "701.314 827 1 (ID: 198)"  "703.290 730 1 (ID: 133)" 
[19] "709.284 697 1 (ID: 155)"  "721.303 945 1 (ID: 159)" 
[21] "725.339 774 1 (ID: 165)"  "739.319 593 1 (ID: 91)"  
[23] "747.321 773 1 (ID: 134)"  "753.332 773 1 (ID: 625)" 
[25] "757.285 870 1 (ID: 223)"  "758.373 1047 1 (ID: 144)"
[27] "758.373 1257 1 (ID: 615)" "761.301 595 1 (ID: 176)" 
[29] "767.345 870 1 (ID: 14)"   "780.353 1045 1 (ID: 616)"
[31] "789.332 870 1 (ID: 34)"   "789.332 1014 1 (ID: 108)"
[33] "793.361 1211 1 (ID: 378)" "802.398 1055 1 (ID: 137)"
[35] "803.307 939 1 (ID: 24)"   "806.335 1061 1 (ID: 31)" 
[37] "820.349 1210 1 (ID: 180)" "824.377 1038 1 (ID: 619)"
[39] "828.318 1067 1 (ID: 210)" "834.368 1463 1 (ID: 620)"
[41] "836.384 1150 1 (ID: 161)" "840.319 1120 1 (ID: 70)" 
[43] "856.348 1463 1 (ID: 611)" "856.348 1245 1 (ID: 643)"
[45] "858.369 1147 1 (ID: 622)" "866.393 1330 1 (ID: 286)"

In [31]:
# Prepare the new table
new <- cbind(new[,1:5], c(0, p.adjust(as.numeric(ct[-c(1:2),2]), method = "bonferroni")), new[,-c(1:5)])
colnames(new)[6] <- "p_value_corrected"
new[,1:10]

shared name,IDs,cor,p_value,Extract,p_value_corrected,F_5,F_6,F_7,F_8
,BioactivityCHIKV,0,0,68.0,0,1.0,4.0,1.0,3.0
1,270.279 1698 1 (ID: 9),-0.324377606725705,0.257849356719369,0.0001391824,1,0.1594079650,0.0228397314,0.3139762067,0.0551088883
2,271.283 1699 1 (ID: 355),-0.318374649942669,0.267271199671765,0.0000139267,1,0.0261136500,0.0037518603,0.0552725554,0.0090100816
3,279.174 1413 1 (ID: 511),0.11344978411662,0.699375059832539,7.760803e-05,1,6.464336e-10,1.221705e-10,8.650469e-10,3.423586e-10
4,280.264 1420 1 (ID: 1029),-0.200702959079237,0.49145327312603,4.456230e-05,1,6.101911e-02,8.258893e-05,8.404894e-05,5.121893e-05
5,281.190 1031 1 (ID: 1070),-0.201330433729144,0.490071225054009,4.369353e-05,1,6.464336e-10,1.194904e-04,2.992650e-02,8.518229e-05
6,282.279 1660 1 (ID: 345),-0.201928970163321,0.48875460960341,4.563084e-04,1,5.618968e-01,1.014960e-03,2.860098e-03,9.171837e-04
7,297.185 1652 1 (ID: 1035),-0.27121389090935,0.348283666308653,6.741464e-05,1,6.464336e-10,1.150745e-02,8.201229e-03,7.540400e-05
8,297.185 1556 1 (ID: 357),-0.214874888224598,0.460688215436271,3.286022e-04,1,6.464336e-10,4.233761e-03,5.458206e-02,4.329827e-04
9,298.311 1968 1 (ID: 132),-0.296642926666024,0.3030698225754,2.040302e-05,1,2.699247e-02,6.353464e-03,7.801129e-02,7.468284e-03


In [32]:
# Write the final table with corrected p_value
write.csv(new, "features_quantification_matrix_transposed_with_significant_correlation_pvalue_corrected.csv", row.names=FALSE)